In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
!pip install cupy-cuda12x

In [9]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [10]:
import cupy as cp

In [11]:
import gensim.downloader as api
dataset = api.load("text8")  # or "text8", "wiki-english-20171001", etc.

In [12]:
sentences = [list(sent) for sent in dataset]
words = 0
for s in sentences:
    words += len(s)

In [28]:
words # 17005207

17005207

In [29]:
flat_words = [word for sentence in sentences for word in sentence]
len(flat_words)

17005207

In [31]:
len(set(flat_words)) # unique words in the complete corpus -> vocab size

253854

In [43]:
c = 0
for w in ("queen", "king", "man", "woman"):
    if w in set(flat_words[:100000]):
        c += 1
    else:
        print(f"{w} not found in corpus")
if c == 4:
    print("all words present int the corpus")

all words present int the corpus


In [48]:
corpus = flat_words[:100000]
vocab = set(flat_words[:100000])
vocab_size = len(vocab)
m = 4 # context window of 9 words [4 future and 4 history and 1 center word]
d = 100 # starting out with 100 dimensionality for word vectors

In [49]:
len(vocab), len(corpus)

(12023, 100000)

In [ ]:
# Init embeddings
U = cp.random.normal(0, 1.0/d, (vocab_size, d))  # Context
V = cp.random.normal(0, 1.0/d, (vocab_size, d))  # Center

In [19]:
V = cp.random.randn(len(vocab), d) * 0.01 # v x d size centre embeddings

In [21]:
U = cp.random.randn(len(vocab), d) * 0.01 # v x d size context 

In [22]:
U

array([[ 0.00520038,  0.00320034,  0.01934578, ...,  0.01067205,
        -0.00542434, -0.00568659],
       [-0.01091278,  0.0035822 ,  0.01596094, ...,  0.01273753,
         0.00255788, -0.00145533],
       [-0.00646176,  0.0167437 ,  0.0133724 , ...,  0.00882345,
         0.00532561, -0.00499894],
       ...,
       [ 0.01278435,  0.00815097,  0.01098265, ...,  0.01263066,
        -0.00862701,  0.00847664],
       [ 0.00464068, -0.01891135,  0.00982112, ..., -0.0058296 ,
        -0.00904889, -0.01955668],
       [ 0.01049755, -0.00044272, -0.0120963 , ..., -0.00302649,
        -0.00413592,  0.017983  ]])

In [23]:
U.shape, V.shape # context, center embeddings.

((10000, 100), (10000, 100))

In [17]:
ctx_w = U[3 - 3 : 3 + 3 + 1, :]
idx = 3
c = ctx_w[idx, :]

In [18]:
k = cp.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
t = cp.array([1, 2, 1])
(cp.dot(k, t))

array([ 8, 20, 32])

In [19]:
c.shape, U.shape

((3,), (7, 3))

In [20]:
ctx_w

array([[-0.00138357,  0.00374866,  0.00495507],
       [-0.00359142,  0.00216887, -0.01636547],
       [ 0.0001955 ,  0.00984135,  0.01044953],
       [ 0.00238707,  0.01222974,  0.01254724],
       [-0.00656655, -0.01849339,  0.00968092],
       [ 0.002286  , -0.00975689, -0.01082154],
       [ 0.01004883, -0.01196735, -0.00015065]])

In [21]:
cp.dot(U, c)

array([ 0.00010471, -0.00018739,  0.00025194,  0.0003127 , -0.00012038,
       -0.00024965, -0.00012426])

In [22]:
cp.exp(cp.dot(U, c))

array([1.00010472, 0.99981263, 1.00025197, 1.00031275, 0.99987963,
       0.99975038, 0.99987575])

In [23]:
(cp.exp(cp.dot(U, c))) / cp.sum(cp.exp(cp.dot(U, c)))

array([0.14287235, 0.14283062, 0.14289339, 0.14290207, 0.1428402 ,
       0.14282173, 0.14283964])

In [24]:
sf_sum = (cp.exp(cp.dot(ctx_w, c))).sum()
((cp.dot(ctx_w, c)/sf_sum)[:3]).sum() + ((cp.dot(ctx_w, c)/sf_sum)[4:]).sum()

array(-4.6431794e-05)

In [25]:
# For all words (inefficient! Negative sampling avoids this)
cp.outer(U, c) # Outer product: [|V|, d]

array([[-3.30267108e-06, -1.69206621e-05, -1.73599497e-05],
       [ 8.94831110e-06,  4.58451189e-05,  4.70353319e-05],
       [ 1.18280951e-05,  6.05991925e-05,  6.21724449e-05],
       [-8.57296773e-06, -4.39221123e-05, -4.50624010e-05],
       [ 5.17725382e-06,  2.65247615e-05,  2.72133869e-05],
       [-3.90655251e-05, -2.00145438e-04, -2.05341535e-04],
       [ 4.66671424e-07,  2.39091005e-06,  2.45298191e-06],
       [ 2.34919950e-05,  1.20357159e-04,  1.23481824e-04],
       [ 2.49437624e-05,  1.27795038e-04,  1.31112802e-04],
       [ 5.69810297e-06,  2.91932417e-05,  2.99511451e-05],
       [ 2.91932417e-05,  1.49566507e-04,  1.53449494e-04],
       [ 2.99511451e-05,  1.53449494e-04,  1.57433289e-04],
       [-1.56748108e-05, -8.03071728e-05, -8.23920761e-05],
       [-4.41450133e-05, -2.26169314e-04, -2.32041033e-04],
       [ 2.31090262e-05,  1.18395085e-04,  1.21468812e-04],
       [ 5.45684182e-06,  2.79571820e-05,  2.86829954e-05],
       [-2.32903907e-05, -1.19324275e-04

In [24]:
epochs = 100 # starting with 100 epochs for now.

In [27]:
loss_history = []

In [32]:
from tqdm import tqdm

In [ ]:
# Use the softmax trick
# Apply gradient clipping
# Apply early stopping

In [ ]:
# training using skip-gram

with tqdm(total=total_iterations, desc="Training Skip-gram") as pbar:
    for epoch in tqdm(range(epochs)):
        
        epoch_loss = 0.0
        num_pairs = 0
        # pick a center word
        for (idx, c) in (enumerate(vocab)):
            # for the context window (valid boundaries) [idx - m, idx + m]
            left_boundary = max(0, idx - m)
            right_boundary = min(len(vocab) - 1, idx + m)
        
            # all the context word indices
            context_indices = [i for i in range(left_boundary, right_boundary + 1) if i != idx] # don't take the center word
        
            # for each context (outside) word in the window 
            for o_idx in context_indices:
        
                u_o = U[o_idx, :] # current context word embedding
                v_c = V[idx, :] # centre word embedding
                
                # calculate p(w | c) softmax summation
                dot_products = cp.dot(U, v_c)
                log_sum_exp = cp.log(cp.sum(cp.exp(dot_products)))  # Log-sum-exp for stability
                current_loss = -dot_products[o_idx] + log_sum_exp
                epoch_loss += current_loss
                num_pairs += 1
                
                exp_dots = cp.exp(dot_products) # TODO : use numerical stability 
                p_w_c = exp_dots / cp.sum(exp_dots) # P(w | c) for each context word [don't use w = c]
        
                # calculate gradients
                vc_grad = -u_o + cp.dot(p_w_c, U) # gradient for center word
                uo_grad = (p_w_c[o_idx] - 1)*v_c # gradient for current context word
                # TODO -> understand why outer is being used and use negative sampling later.
                uw_grad = cp.outer(p_w_c, v_c) # gradient for unobserved (leaving the current context word) context words
                # adjust gradient for observed word o
                uw_grad[o_idx, :] -= v_c  # Equivalent to (p(o|c) - 1) * v_c
                
                # Update the parameter vectors using the gradients
                V[idx, : ] -= lr * (vc_grad) # update the center word
                U[o_idx, : ] -= lr * (uo_grad) # update the current (observed) context word
                U -= lr * (uw_grad) # update the unobserved context words
                
                
        avg_epoch_loss = float(epoch_loss / num_pairs)  # Convert to Python float
        loss_history.append(avg_epoch_loss)
        print(f"Epoch {epoch}, Loss: {avg_epoch_loss}")

pbar.close()

In [34]:
# total_iterations = epochs * len(vocab)  # Approximate total pairs
# with tqdm(total=total_iterations, desc="Training Skip-gram") as pbar:
#     for epoch in range(epochs):
#         epoch_loss = 0.0
#         num_pairs = 0
#         for idx in range(len(vocab)):
#             left_boundary = max(0, idx - m)
#             right_boundary = min(len(vocab) - 1, idx + m)
#             context_indices = [i for i in range(left_boundary, right_boundary + 1) if i != idx]
#             for o_idx in context_indices:
#                 u_o = U[o_idx, :]  # context embedding for o
#                 v_c = V[idx, :]    # center embedding for c
#                 dot_products = cp.dot(U, v_c)
#                 log_sum_exp = cp.log(cp.sum(cp.exp(dot_products - cp.max(dot_products))) + cp.max(dot_products))
#                 current_loss = -cp.dot(u_o, v_c) + log_sum_exp
#                 epoch_loss += current_loss
#                 num_pairs += 1
                
#                 exp_dots = cp.exp(dot_products - cp.max(dot_products))
#                 p_w_c = exp_dots / cp.sum(exp_dots)
                
#                 vc_grad = -u_o + cp.dot(p_w_c, U)
#                 uo_grad = (p_w_c[o_idx] - 1) * v_c
#                 uw_grad = p_w_c * v_c
#                 uw_grad[o_idx] = (p_w_c[o_idx] - 1) * v_c
                
#                 V[idx, :] -= lr * vc_grad
#                 U[o_idx, :] -= lr * uo_grad
#                 for w in range(len(vocab)):
#                     U[w, :] -= lr * uw_grad[w]
                
#                 pbar.update(1)  # Update progress bar per pair
        
#         avg_epoch_loss = float(epoch_loss / num_pairs)
#         loss_history.append(avg_epoch_loss)
#         pbar.set_postfix({'Epoch Loss': avg_epoch_loss})
#         print(f"Epoch {epoch}, Loss: {avg_epoch_loss}")

# pbar.close()

Training Skip-gram:   0%|          | 0/1000000 [00:00<?, ?it/s]


ValueError: operands could not be broadcast together with shapes (10000,) (100,)

In [30]:
V

array([[ 1.40136926, -0.49552651, -0.42532298],
       [ 0.05313453, -0.03351528,  0.68138598],
       [-0.23847167, -0.40992499, -0.06977735],
       [-0.09327522,  0.22551045, -0.07777905],
       [ 0.29441731, -0.07724471, -0.13224349],
       [ 0.32116448,  0.18530075,  1.10928156],
       [-0.60548336, -1.4123885 ,  0.1138442 ]])

In [31]:
vocab

['The', 'cat', 'and', 'dog', 'had', 'a', 'fight']